In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import r2_score, explained_variance_score

# cross validation purposes: create the cartesian product between the chosen values sets
from itertools import product 

#import os
#import seaborn as sns
#import matplotlib.pyplot as plt
#%matplotlib inline

# Read Dataset

In [2]:
cmp = pd.read_csv("commViolUnnormData.txt", na_values='?')

In [3]:
# drop first non predictive features (communityname, state, countyCode, communityCode, "fold")
pred_features = cmp[cmp.columns[5:-18]]
regr_values = cmp[cmp.columns[-18:]]

# Drop features with a lot of missing values

In [4]:
print("Before dropping: {} features".format(str(pred_features.shape[1])))

#drop features that contain at least some threshold (from the total) of NaN values
cut_tresh = 0.75
to_drop = pred_features.columns[pred_features.count() < pred_features.shape[0]*cut_tresh]

pred_features = pred_features.drop(columns=to_drop)

print("After dropping: {} features".format(str(pred_features.shape[1])))

Before dropping: 124 features
After dropping: 102 features


# Imputing on features matrix

In [5]:
from collections import Counter

def value_withStrategy(v, strat):
    if strat == "mean":
        return np.mean(v)
    if strat == "median":
        return np.median(v)
    if strat == "most_frequent":
        return Counter(v).most_common(1)[0][0]
    print("Invalid imputing strategy!")
        
def imputing(df, strategy):
    # for each column that contain at least 1 NaN value...
    for nanCol in np.unique(np.where(pred_features.isna())[1]):
        nanRows = np.where(pred_features.iloc[:,nanCol].isna())[0] #find NaN rows for the current column
        available = df.iloc[~nanRows, nanCol]
        value = value_withStrategy(available, strategy) #compute the filling value
        df.iloc[nanRows, nanCol] = value

In [6]:
imputing(pred_features, "mean");

# Choose the Dependent Variable and drop possible missing values on it

In [7]:
def drop_sample(df, vals):
    idxRow = np.where(vals.isna())[0]
    return df.drop(index=idxRow).values, vals.drop(index=idxRow).values.reshape(-1,1)

In [8]:
data,values = drop_sample(pred_features, regr_values["robbPerPop"])

# Normalisation

In [9]:
def normalise(matrix, strat):
    for j in range(matrix.shape[1]):
        mi = np.min(matrix[:,j])
        ma = np.max(matrix[:,j])
        di = ma-mi
        if (di > 1e-6):
            if strat=="0_mean,1_std":
                matrix[:,j] = (matrix[:,j]-np.mean(matrix[:,j]))/np.std(matrix[:,j])
            elif strat=="[0,1]":
                matrix[:,j] = (matrix[:,j]-mi)/di
            elif strat=="[-1,1]":
                matrix[:,j] = 2*((matrix[:,j]-mi)/di)-1
            else:
                print("Invalid normalisation strategy!")
        else:
            matrix[:,j] = 0

In [10]:
strategy = "[-1,1]"
normalise(data,strategy)
normalise(values,strategy)

# Train-Test Split

In [11]:
def trainTest_split(in_matrix, out_vect, train_amount=0.7):
    n = in_matrix.shape[0]

    trVl_Amount = int(n*train_amount) #training-validation amount
    indexes = np.random.permutation(n)
    idxTrVl = np.sort(indexes[0:trVl_Amount])
    idxTs = np.sort(indexes[trVl_Amount:])

    return in_matrix[idxTrVl], in_matrix[idxTs], out_vect[idxTrVl], out_vect[idxTs]

In [12]:
trainVal_data, test_data, trainVal_values, test_values = trainTest_split(data, values, train_amount=0.7)

# Variable Selection

## 0. K-fold Cross Validation

In [13]:
from itertools import product

def kFold_crossValidation_selectionGrid(k, parameters_dict, train_data, train_values, predictor, verbose=False):
    nVal = train_data.shape[0]
    
    # Validation indexes adjustment -------------------------------
    elemPerFold, remainder = np.divmod(nVal,k) #the remainder will be distributed across the firsts folds
    valIdxList = []
    start = 0

    # in each fold put as many samples as the division quotient +1 if the remainder is still positive
    # then decrease the division remainder by 1
    for i in range(k): 
        end = start+elemPerFold+int(remainder>0)
        valIdxList.append(np.arange(start,end)) 
        remainder -= 1
        start = end
    
    # Cross validation --------------------------------------------
    params_names = parameters_dict.keys()
    params_product = list(product(*parameters_dict.values())) # build all the hyp-par combination
    val_results = np.empty((len(valIdxList),len(params_product)))
    
    for row, valIdx in enumerate(valIdxList): # for each fold
        if verbose: print("#{} fold:".format(row+1))
        for col, params in enumerate(params_product):
            
            if verbose:
                update = col*100/len(params_product) # just print completion rate
                print("\t["+"#"*(int(update/5))+" "*(int((100-update)/5))+"] {}%".format(update))
                     
            arg_dict = {k:v for k,v in zip(params_names,params)} # {argument_name:argument_value, ... }
            
            
            predictor.fit(train_data[~valIdx], train_values[~valIdx], **arg_dict)
            pred = predictor.predict(train_data[valIdx])
            
            #val_results[row,col] = r2_score(trainVal_values[valIdx],pred)
            #val_results[row,col] = explained_variance_score(trainVal_values[valIdx],pred)
            val_results[row,col] = np.mean(np.square(train_values[valIdx]-pred)) # MSE
            
    selected = np.argmin(val_results.mean(axis=0))
    return params_product[selected]

## 1. Matching Pursuit

### Project class definition

In [14]:
class matchingPursuit:
    def __init__(self, iterations, weights = None, indexes = None):
        self.iterations = iterations
        self.weights = weights
        self.indexes = indexes
        
    def fit(self, data_matrix, output_vect):
        residual = output_vect.copy()
        self.weights = np.zeros((data_matrix.shape[1], 1))
        self.indexes = []

        #data_2norm = np.sqrt(np.sum(np.square(data_matrix), axis=0))
        data_2norm = np.linalg.norm(data_matrix, ord=2, axis=0).reshape(1,-1)

        for i in range(self.iterations):
            
            # project each column on the current residuals
            projection = np.matmul(residual.T, data_matrix)
            # find the most correlated variable
            k = np.argmax(np.divide(np.square(projection), data_2norm))
            self.indexes.append(k)
            
            distance = projection[0,k]/np.linalg.norm(data_matrix[:,k], ord=2)
            self.weights[k,0] += distance # update the solution vector: canonical basis over the found column
            residual -= np.matmul(data_matrix, self.weights) # update the residual

        return self
    
    def predict(self, X):
        if self.weights is None:
            raise Exception("weights not initialised! need to first fit the model")
        return np.matmul(X, self.weights)

In [15]:
mp = matchingPursuit(iterations=10)
mp.fit(trainVal_data, trainVal_values)
np.where(mp.weights)[0]

array([92])

In [16]:
pred = mp.predict(test_data)
print("Mean Square Error: {}".format(np.mean(np.square(test_values-pred))))
print("R^2 score: {}".format(r2_score(test_values, pred)))
print("Explained Variance Score: {}".format(explained_variance_score(test_values, pred)))

Mean Square Error: 2.1477284527146093e+31
R^2 score: -5.2554165574405774e+32
Explained Variance Score: -4.480652353657181e+29


### SkLearn Class

In [17]:
from sklearn.linear_model import orthogonal_mp
omp_coef = orthogonal_mp(trainVal_data, trainVal_values)
np.where(omp_coef)[0]

array([ 3, 11, 34, 38, 50, 76, 77, 92, 93, 94])

In [18]:
pred = np.matmul(test_data, omp_coef)
print("Mean Square Error: {}".format(np.mean(np.square(test_values-pred))))
print("R^2 score: {}".format(r2_score(test_values, pred)))
print("Explained Variance Score: {}".format(explained_variance_score(test_values, pred)))

Mean Square Error: 0.06847062493726377
R^2 score: 0.6652543463899951
Explained Variance Score: 0.6654209600448748


## 2. L1 Penalty (Lasso)

### Project class definition

In [19]:
class lasso_regression: # Iterative Soft Thresholding Algorithm (Proximal Gradient)
    def __init__(self, iterations, weights=None):
        self.iterations = iterations
        self.weights = weights
        
    def fit(self, data_matrix, output_vect, _lambda):
        self.weights = np.zeros((data_matrix.shape[1],1))
        n = data_matrix.shape[0]
        # convergence step-size: n/(2*||X^t*X||_2)
        step = n/(2*np.linalg.norm(np.matmul(data_matrix.T, data_matrix), ord=2))
        softTresh = step*_lambda

        for i in range(self.iterations):
            # gradient step of the lasso formulation
            dist = np.matmul(data_matrix, self.weights) - output_vect
            coord_descent = (step/n)*np.matmul(data_matrix.T, dist)
            self.weights -= coord_descent

            # soft thresholding operator
            upper = self.weights > softTresh  # elem to be reduced
            lower = self.weights < -softTresh # elem to be increased
            self.weights[upper] -= softTresh
            self.weights[lower] += softTresh
            self.weights[~upper & ~lower] = 0

        return self
    
    def predict(self, X):
        if self.weights is None:
            raise Exception("weights not initialised! need to first fit the model")
        return np.matmul(X, self.weights)

In [20]:
lr = lasso_regression(iterations=10)
lr.fit(trainVal_data, trainVal_values, 0.8)
np.where(lr.weights)[0]

array([  0,  10,  27,  49,  51,  71,  91,  92,  98, 101])

In [21]:
pred = lr.predict(test_data)
print("Mean Square Error: {}".format(np.mean(np.square(test_values-pred))))
print("R^2 score: {}".format(r2_score(test_values, pred)))
print("Explained Variance Score: {}".format(explained_variance_score(test_values, pred)))

Mean Square Error: 0.7262956170196937
R^2 score: -16.772200235357317
Explained Variance Score: 0.005051931736064108


### SkLearn Class

In [22]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.005)
lasso.fit(trainVal_data, trainVal_values)
np.where(lasso.coef_)[0]

array([  3,  11,  38,  44,  50,  76,  93,  94, 100])

In [23]:
pred = lasso.predict(test_data)
print("Mean Square Error: {}".format(np.mean(np.square(test_values-pred))))
print("R^2 score: {}".format(r2_score(test_values, pred)))
print("Explained Variance Score: {}".format(explained_variance_score(test_values, pred)))

Mean Square Error: 0.06121756805249363
R^2 score: 0.6328578269018734
Explained Variance Score: 0.6335227522165594


## 3. Random Forest

### Decision Tree project class definition

In [24]:
class NumericalDecisionTree_regressor:
    class Node:
        def __init__(self, value, isLeaf=False, feature=None, left=None, right=None):
            self.value = value
            self.isLeaf = isLeaf
            self.feature = feature
            self.left = left
            self.right = right

        def print_tree(self):
            if self.left: self.left.print_tree()
            print("Feature: {}, cut: {}\n".format(self.feature, self.value))
            if self.right: self.right.print_tree()

        def print_tree_indented(self, level=0):
            if self.right: self.right.print_tree_indented(level+1)
            print("|    "*level+"{} => {}".format(self.feature, self.value))
            if self.left: self.left.print_tree_indented(level+1)
            
    def __init__(self):
        self.root = None
        
    def fit(self, X, y, depth, minElem_perLeaf):
        self.root = self.learn(X, y, depth, minElem_perLeaf)
        return self
        
    def learn(self, X, y, depth, minElem_perLeaf):
        n, d = X.shape

        if depth==0 or n<=minElem_perLeaf: # leaf
            return self.Node(value=np.mean(y), isLeaf=True)
            
        best_costDescent = 0 # split that maximise the error descent

        for i1 in range(d):
            sorted_idx = np.argsort(X[:,i1])
            sorted_x, sorted_y = X[sorted_idx, i1], y[sorted_idx]

            s_right, s_left = np.sum(sorted_y), 0
            n_right, n_left = n, 0

            for i2 in range(n-1):
                s_left += sorted_y[i2]
                s_right -= sorted_y[i2]
                n_left += 1
                n_right -= 1
                
                if sorted_x[i2]<sorted_x[i2+1]: # for a different value
                    # try to maximise this value: it is directly correlated 
                    # to the possible split information gain
                    new_costDescent = (s_left**2)/n_left + (s_right**2)/n_right
                    if new_costDescent > best_costDescent:
                        best_costDescent = new_costDescent
                        best_feature = i1
                        best_cut = (sorted_x[i2]+sorted_x[i2+1])/2

        left_idxs = X[:,best_feature] < best_cut

        return self.Node(value=best_cut, feature=best_feature,
                        left=self.learn(X[left_idxs],y[left_idxs],depth-1,minElem_perLeaf),
                        right=self.learn(X[~left_idxs],y[~left_idxs],depth-1,minElem_perLeaf))
    
    def predict(self, X):
        if self.root is None:
            raise Exception("Tree not initialised! need to first fit the model")

        n = X.shape[0]
        y = np.empty(n)
        
        for i in range(n):
            current = self.root
            while not current.isLeaf:
                if X[i,current.feature] < current.value:
                    current = current.left
                else:
                    current = current.right
                
            y[i] = current.value
        
        return y
                
    def pprint(self):
        self.root.print_tree_indented()

In [25]:
ndt = NumericalDecisionTree_regressor()
ndt.fit(trainVal_data, trainVal_values, depth=5, minElem_perLeaf=10)
ndt.pprint()

|    |    |    |    None => 0.8259154730514591
|    |    |    61 => -0.45812679363422903
|    |    |    |    |    None => 0.29717059406609014
|    |    |    |    100 => -0.3042517945886251
|    |    |    |    |    None => -0.08725058484568764
|    |    50 => -0.09104204753199274
|    |    |    |    |    None => 0.03270277472288843
|    |    |    |    5 => -0.7431963854155721
|    |    |    |    |    None => -0.2864708046168475
|    |    |    74 => -0.809726748558536
|    |    |    |    None => -0.6061568902845685
|    100 => -0.5549420209828824
|    |    |    |    |    None => -0.08184993303780658
|    |    |    |    89 => -0.16042780748663116
|    |    |    |    |    None => -0.441234339354147
|    |    |    40 => 0.3033543996110841
|    |    |    |    |    None => -0.49114670977373215
|    |    |    |    61 => -0.9668666840594834
|    |    |    |    |    None => -0.6937619053928631
|    |    50 => -0.48957952468007315
|    |    |    |    |    None => -0.7156861446066415
|    |    |  

In [26]:
pred = ndt.predict(test_data)
print("Mean Square Error: {}".format(np.mean(np.square(test_values-pred))))
print("R^2 score: {}".format(r2_score(test_values, pred)))
print("Explained Variance Score: {}".format(explained_variance_score(test_values, pred)))

Mean Square Error: 0.0638563134391556
R^2 score: 0.5709126687760459
Explained Variance Score: 0.5715468520381957


### Decision Tree SkLearn Class

In [27]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor()
dtr.fit(trainVal_data, trainVal_values)
np.flip(np.argsort(dtr.feature_importances_))

array([ 49, 100,   3,  50,  14,  40,  89,  29,  98,  74,  41,  61,  88,
         0,  36,  68,  18,  79,  95,  99,   4,  35,   9,  45,  92,  72,
        15,  38,  12,  39,  24,   2,  78,  77,  19,  52,  62,  30,  66,
        85,  33,  34,  25,  73,  23,  11,  90,   6,  55,  94,  69,  76,
         1,  83,  46,  53,  44,  93,  57,  27,  42,  48,  16,  71,   5,
        64,  58,  13,  28,  22,   8,  75,  51,  20,  32,  17,  43,  96,
        70,  84,  86,  81,  37,  59,  67,  26,  80,  47,   7,  54,  97,
        65,  63,  87,  21,  31,  10,  91,  82,  56, 101,  60])

In [28]:
pred = dtr.predict(test_data)
print("Mean Square Error: {}".format(np.mean(np.square(test_values-pred))))
print("R^2 score: {}".format(r2_score(test_values, pred)))
print("Explained Variance Score: {}".format(explained_variance_score(test_values, pred)))

Mean Square Error: 0.07299162980474981
R^2 score: 0.5520523728524112
Explained Variance Score: 0.5521896252959912


### Random Forest project class definition

In [29]:
#https://stackoverflow.com/questions/18541923/what-is-out-of-bag-error-in-random-forests

class NumericalRandomForest_regressor:
    def __init__(self, n_trees):
        self.n_trees = n_trees
        self.trees = []
        self.boot_samplesIdxs = []
        self.oob_error = None
        
    def fit(self, X, y, depths, minElems_perLeaf):
        n = X.shape[0]
        n_learn = int(n/3) # Bootstrap amount to be taken aside
        
        val_folds = 3
        params_dict = {"depth":depths, "minElem_perLeaf":minElems_perLeaf}
        
        # Fitting the forest -----------------------------------
        for i in range(self.n_trees):
            print("Fitting #{} tree".format(i+1))
            
            bootstrap_idxs = np.sort(np.random.permutation(n)[:n_learn])
            self.boot_samplesIdxs.append(bootstrap_idxs)
                        
            dt = NumericalDecisionTree_regressor()
            # find the best hyp-par for the current setting (bootstrapping)
            win_params = kFold_crossValidation_selectionGrid(val_folds, params_dict, 
                                                             X[~bootstrap_idxs], y[~bootstrap_idxs],
                                                             dt, verbose=True)
            self.trees.append(dt.fit(X[~bootstrap_idxs], y[~bootstrap_idxs],
                                     depth=win_params[0], minElem_perLeaf=win_params[1]))
        
        # Out-Of-Bag Estimate for the forest -------------------
        oob_errors = []
        for sampleIdx in range(n):
            missingBoot_TreesIdx = [idx for idx,bootstrap_idxs in enumerate(self.boot_samplesIdxs) 
                                    if sampleIdx not in bootstrap_idxs]

            if len(missingBoot_TreesIdx) == 0: continue
            
            regr_results = np.empty(len(missingBoot_TreesIdx)) # regression estimate of the selected trees
            for i, missing_tree in enumerate(missingBoot_TreesIdx):
                regr_results[i] = self.trees[missing_tree].predict(X[sampleIdx].reshape(1,-1))
            
            # done at this level of granularity because a sample might end up in 
            # being part of no bootstrap set of any tree (so we cannot predict wich value in y will be used)
            oob_errors.append(np.mean(np.square(np.mean(regr_results)-y[sampleIdx])))
            #oob_errors.append(r2_score(np.mean(regr_results),y[sampleIdx]))
            #oob_errors.append(explained_variance_score(np.mean(regr_results),y[sampleIdx]))
            
        self.oob_error = np.mean(oob_errors)
        return self
            
        
    def predict(self,X):
        if len(self.trees)==0:
            raise Exception("trees not initialised! need to first fit the model")

        n = X.shape[0]
        results = np.empty((self.n_trees,n))
        for row, tree in enumerate(self.trees):
            results[row] = tree.predict(X)
            
        return np.mean(results,axis=0)

In [30]:
nrf = NumericalRandomForest_regressor(3)
# no train and test, cause it's a forest
nrf.fit(data,values,depths=[10,20],minElems_perLeaf=[20,30]);
nrf.oob_error

Fitting #1 tree
#1 fold:
	[                    ] 0.0%
	[#####               ] 25.0%
	[##########          ] 50.0%
	[###############     ] 75.0%
#2 fold:
	[                    ] 0.0%
	[#####               ] 25.0%
	[##########          ] 50.0%
	[###############     ] 75.0%
#3 fold:
	[                    ] 0.0%
	[#####               ] 25.0%
	[##########          ] 50.0%
	[###############     ] 75.0%
Fitting #2 tree
#1 fold:
	[                    ] 0.0%
	[#####               ] 25.0%
	[##########          ] 50.0%
	[###############     ] 75.0%
#2 fold:
	[                    ] 0.0%
	[#####               ] 25.0%
	[##########          ] 50.0%
	[###############     ] 75.0%
#3 fold:
	[                    ] 0.0%
	[#####               ] 25.0%
	[##########          ] 50.0%
	[###############     ] 75.0%
Fitting #3 tree
#1 fold:
	[                    ] 0.0%
	[#####               ] 25.0%
	[##########          ] 50.0%
	[###############     ] 75.0%
#2 fold:
	[                    ] 0.0%
	[#####           

0.011414943558457733

### Random Forest SkLearn class

In [32]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
rfr.fit(trainVal_data, trainVal_values.ravel())
np.flip(np.argsort(rfr.feature_importances_))

/Users/danilo/.conda/envs/bcb/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


array([ 50,  49, 100,   3,  92,  69,  62,  51,  15,  38,   2,  14,  46,
        99,  40,  63,  71,   4,  94,   0,  35,  39,  37,  28,  68,  82,
        41,  44,  22,  67,  75,  43,  90,  18,  98,  20,  52,  47,   5,
        86,  21,  88,   6,  64,  23,  58,  16,  24,  29,  60, 101,  73,
         8,  76,  97,  95,  34,  78,  17,   9,  10,  25,  96,  59,  89,
        32,  72,  45,  66,  79,  74,  48,  87,   7,  31,  33,  30,  83,
        26,  61,  84,  93,  65,  36,  77,  27,  85,  13,  11,   1,  80,
        19,  55,  54,  42,  53,  91,  56,  57,  12,  81,  70])

In [33]:
pred = rfr.predict(test_data)
print("Mean Square Error: {}".format(np.mean(np.square(test_values-pred))))
print("R^2 score: {}".format(r2_score(test_values, pred)))
print("Explained Variance Score: {}".format(explained_variance_score(test_values, pred)))

Mean Square Error: 0.07115004850034741
R^2 score: 0.7041212016715361
Explained Variance Score: 0.7063583679107304


# Predictors

## 1. Regularised Least Squares
   

In [34]:
class tikhonov_leastSquares:
    def __init__(self, weights = None):
        self.weights = weights
        
    def fit(self, X, y, _lambda):
        inv = np.linalg.inv(np.matmul(X.T, X) + _lambda*np.eye(X.shape[1]))
        self.weights = np.matmul(inv, np.matmul(X.T, y))
        return self
    
    def predict(self, X):
        if self.weights is None:
            raise Exception("weights not initialised! need to first fit the model")
        return np.matmul(X, self.weights)

In [35]:
k = 5
params_dict = {"_lambda":[2,2.05,2.1,2.2,3]}

tls = tikhonov_leastSquares()

win_regulariser = kFold_crossValidation_selectionGrid(k, params_dict, trainVal_data, trainVal_values, tls)
tls.fit(trainVal_data, trainVal_values, win_regulariser)
pred = tls.predict(test_data)

print("Mean Square Error: {}".format(np.mean(np.square(test_values-pred))))
print("R^2 score: {}".format(r2_score(test_values, pred)))
print("Explained Variance Score: {}".format(explained_variance_score(test_values, pred)))

Mean Square Error: 0.010942369403192055
R^2 score: 0.7322440401323482
Explained Variance Score: 0.7322662657251657


## 2. Random Forest

In [37]:
rf = NumericalRandomForest_regressor(5)
rf.fit(trainVal_data, trainVal_values, depths=[10,20,30], minElems_perLeaf=[20,30,40]);

pred = rf.predict(test_data)
print("Mean Square Error: {}".format(np.mean(np.square(test_values-pred))))
print("R^2 score: {}".format(r2_score(test_values, pred)))
print("Explained Variance Score: {}".format(explained_variance_score(test_values, pred)))

Fitting #1 tree
#1 fold:
	[                    ] 0.0%
	[##                 ] 11.11111111111111%
	[####               ] 22.22222222222222%
	[######             ] 33.333333333333336%
	[########           ] 44.44444444444444%
	[###########        ] 55.55555555555556%
	[#############      ] 66.66666666666667%
	[###############    ] 77.77777777777777%
	[#################  ] 88.88888888888889%
#2 fold:
	[                    ] 0.0%
	[##                 ] 11.11111111111111%
	[####               ] 22.22222222222222%
	[######             ] 33.333333333333336%
	[########           ] 44.44444444444444%
	[###########        ] 55.55555555555556%
	[#############      ] 66.66666666666667%
	[###############    ] 77.77777777777777%
	[#################  ] 88.88888888888889%
#3 fold:
	[                    ] 0.0%
	[##                 ] 11.11111111111111%
	[####               ] 22.22222222222222%
	[######             ] 33.333333333333336%
	[########           ] 44.44444444444444%
	[###########        ] 55.5

In [38]:
rf.oob_error

0.011831909775350221